# Welcome to the generative Mistral demo 🥳

In this demo, we will use Cohere to generate the embeddings for the blog post and use Mistral's language model to generate content!

What you will need for this demo:
1. A Weaviate cluster (more info below)
2. Cohere API key 
3. Mistral API key

## Dependencies

In [ ]:
!pip install weaviate-client

## Connect to Weaviate Cluster

You can create a free 14-day sandbox on [WCS](https://console.weaviate.cloud/signin), or learn about our other installation options [here](https://weaviate.io/developers/weaviate/installation)!

In [ ]:
import weaviate
import os

# Connect to a WCS instance
client = weaviate.connect_to_wcs(
    cluster_url=("cluster-url"),  # URL to your sandbox
    auth_credentials=weaviate.auth.AuthApiKey("auth-key"), # api key to your sandbox if you set up auth
    headers= {'X-Cohere-API-Key': "key", #cohere key
    'X-Mistral-Api-key': "key" # mistral key
   }
)

## Create schema

In [ ]:
# CAUTION: This will delete your schema and the objects!
# only run the below if you need to reset your schema

# client.collections.delete_all()

In [ ]:
import weaviate.classes.config as wvcc
from weaviate.classes.config import Property, DataType


collection = client.collections.create(
    name="BlogChunks", # name of the collection
    vectorizer_config=wvcc.Configure.Vectorizer.text2vec_cohere
    (
        model="embed-multilingual-v3.0" # Cohere model (we support other models in Weaviate)
    ),
    generative_config=wvcc.Configure.Generative.mistral(
        model="open-mistral-7b" # Mistral model (we support other models in Weaviate)
    ),
    properties=[
            Property(name="content", data_type=DataType.TEXT) # We only have one property for our collection. It is the content within the blog posts
    ]
)

## Chunk and Import Data

We need to break our blog posts into smaller chunks

In [ ]:
import os
import re

def chunk_list(lst, chunk_size):
    """Break a list into chunks of the specified size."""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

def split_into_sentences(text):
    """Split text into sentences using regular expressions."""
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

def read_and_chunk_index_files(main_folder_path):
    """Read index.md files from subfolders, split into sentences, and chunk every 5 sentences."""
    blog_chunks = []
    
    for file_path in os.listdir("./data"):
        index_file_path = os.path.join("./data", file_path)
        with open(index_file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            sentences = split_into_sentences(content)
            sentence_chunks = chunk_list(sentences, 5)
            sentence_chunks = [' '.join(chunk) for chunk in sentence_chunks]
            blog_chunks.extend(sentence_chunks)
    return blog_chunks

# run with:
main_folder_path = './data'
blog_chunks = read_and_chunk_index_files(main_folder_path)

In [ ]:
# First chunk

blog_chunks[0]

In [ ]:
# Insert the objects (chunks) into the Weaviate cluster

from weaviate.util import get_valid_uuid
from uuid import uuid4
blogs = client.collections.get("BlogChunks")

for blog_chunk in blog_chunks:
    random_uuid = get_valid_uuid(uuid4())
    blogs.data.insert(
        properties={
            "content": blog_chunk
        },
        uuid=random_uuid
    )


## Query Time 

## Hybrid Search Query

Hybrid search combines BM25 and vector search and weighs the two algorithms depending on the `alpha` parameter. 

`alpha`= 0 --> pure BM25

`alpha`= 0.5 --> half BM25, half vector search

`alpha`= 1 --> pure vector search

In [ ]:
import json 

blogs = client.collections.get("BlogChunks")

response = blogs.query.hybrid(
    query="What is Ref2Vec",
    alpha=0.5,
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

### Generative Search Query

Here is what happens in the below:
1. We will retrieve 3 relevant chunks from our vector database (powered by the Cohere embeddings)
2. We will pass the 3 chunks to Mistral to generate the short paragraph about Ref2Vec

The first line in the output is the generated text, and the `content` pieces below it, are what was retrieved from Weaviate and passed to Mistral.

In [ ]:
blogs = client.collections.get("BlogChunks")


response = blogs.generate.near_text(
    query="What is Ref2Vec?",
    single_prompt="Write a short paragraph about ref2vec with this content: {content}",
    limit=3
)


for o in response.objects:
    print(o.generated)
    print(json.dumps(o.properties, indent=2))